<a href="https://colab.research.google.com/github/anjali88/Basic-ML-/blob/master/MarketRisk_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

# https://github.com/hamaadshah/market_risk_gan_keras/blob/master/Python/market_risk.ipynb

from PIL import Image

from six.moves import range

import os
import math
import inspect
import sys
import importlib

import numpy as np

import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import keras
from keras import backend as bkend
from keras import layers
from keras.layers import Input, Dense, BatchNormalization, Dropout, Flatten, convolutional, pooling, Reshape, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras import metrics
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras.utils.generic_utils import Progbar

import tensorflow as tf
from tensorflow.python.client import device_lib

import matplotlib.pyplot as plt

from plotnine import *
import plotnine

get_ipython().magic("matplotlib inline")

os.environ["KERAS_BACKEND"] = "tensorflow"
importlib.reload(bkend)

print(device_lib.list_local_devices())


Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13527669937339327129
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15214655053433735538
physical_device_desc: "device: XLA_CPU device"
]


In [3]:
#upload dataset
from google.colab import files
uploaded = files.upload()

Saving data_second.xlsx to data_second.xlsx


In [20]:
import io
ret_data = pd.read_excel(io.BytesIO(uploaded['data_second.xlsx']))
# Dataset is now stored in a Pandas Dataframe
#print the head
ret_data.head()


,Dates,Open,Close,High,Low,Value,Volume
0,2018-02-14 08:00:00,1682.0,1679.8,1682.0,1677.6,36935376.0,21978
1,2018-02-14 08:01:00,1679.8,1680.8,1681.6,1678.0,28434078.0,16930
2,2018-02-14 08:02:00,1681.2,1679.0,1681.2,1678.0,24295556.0,14466
3,2018-02-14 08:03:00,1679.0,1675.6,1679.8,1675.4,19602172.0,11683
4,2018-02-14 08:04:00,1674.6,1676.4,1677.4,1674.6,10772108.0,6427


In [0]:
def parse(x):
#function for parsing data into required format
  return datetime.strptime(x, '%Y %m %d %H')

In [0]:
ret_data.drop('Dates', axis=1, inplace=True)

In [23]:
ret_data.head()

,Open,Close,High,Low,Value,Volume
0,1682.0,1679.8,1682.0,1677.6,36935376.0,21978
1,1679.8,1680.8,1681.6,1678.0,28434078.0,16930
2,1681.2,1679.0,1681.2,1678.0,24295556.0,14466
3,1679.0,1675.6,1679.8,1675.4,19602172.0,11683
4,1674.6,1676.4,1677.4,1674.6,10772108.0,6427


In [0]:
ret_data.drop('year', axis=1, inplace=True)
ret_data.drop('month', axis=1, inplace=True)
ret_data.drop('day', axis=1, inplace=True)
ret_data.drop('hour', axis=1, inplace=True)
ret_data.drop('No', axis=1, inplace=True)

In [0]:
mean = ret_data.apply(func=np.mean, axis=0)
ret_data -= mean
std = ret_data.apply(func=np.std, axis=0)
ret_data /= std

In [32]:
class BiGAN(BaseEstimator,
            TransformerMixin):
    def __init__(self,
                 z_size=None,
                 iterations=None,
                 batch_size=None):
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")
        
        for arg, val in values.items():
            setattr(self, arg, val)
            # Build the discriminator.
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(optimizer=RMSprop(lr=0.0002, 
                                                     clipvalue=1.0,
                                                     decay=1e-8),
                                   loss="binary_crossentropy",
                                   metrics=["accuracy"])

        # Build the generator to fool the discriminator.
        # Freeze the discriminator here.
        self.discriminator.trainable = False
        self.generator = self.build_generator()
        self.encoder = self.build_encoder()
        
        noise = Input(shape=(self.z_size, ))
        generated_data = self.generator(noise)
        fake = self.discriminator([noise, generated_data])

        real_data = Input(shape=(5,))
        encoding = self.encoder(real_data)
        valid = self.discriminator([encoding, real_data])

        # Set up and compile the combined model.
        # Trains generator to fool the discriminator.
        self.bigan_generator = Model([noise, real_data], [fake, valid])
        self.bigan_generator.compile(loss=["binary_crossentropy", "binary_crossentropy"],
                                     optimizer=RMSprop(lr=0.0004, 
                                                       clipvalue=1.0,
                                                       decay=1e-8))
        def fit(self,
            X,
            y=None):
         num_train = X.shape[0]
         start = 0
        
        # Adversarial ground truths.
        valid = np.ones((self.batch_size, 1)) 
        fake = np.zeros((self.batch_size, 1))      
        for step in range(self.iterations):
            # Generate a new batch of noise...
            noise = np.random.uniform(low=-1.0, high=1.0, size=(self.batch_size, self.z_size))
            # ...and generate a batch of synthetic returns data.
            generated_data = self.generator.predict(noise)
            
            # Get a batch of real returns data...
            stop = start + self.batch_size
            real_batch = X[start:stop]
            # ...and encode them.
            encoding = self.encoder.predict(real_batch)

            # Train the discriminator.
            d_loss_real = self.discriminator.train_on_batch([encoding, real_batch], valid)
            d_loss_fake = self.discriminator.train_on_batch([noise, generated_data], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train the generator.
            g_loss = self.bigan_generator.train_on_batch([noise, real_batch], [valid, fake])
            
            start += self.batch_size
            if start > num_train - self.batch_size:
                start = 0
            
            if step % 100 == 0:
                # Plot the progress.
                print("[Discriminator loss: %f, Discriminator accuracy: %.2f%%] [Generator loss: %f]" % (d_loss[0], 100 * d_loss[1], g_loss[0]))
                return self

    def transform(self,
                  X):
        return self.feature_extractor.predict(X)

    def build_encoder(self):
        encoder_input = Input(shape=(5,))

        encoder_model = Dense(units=100)(encoder_input)
        encoder_model = LeakyReLU(alpha=0.2)(encoder_model)
        encoder_model = BatchNormalization()(encoder_model)
        encoder_model = Dense(units=100)(encoder_model)
        encoder_model = LeakyReLU(alpha=0.2)(encoder_model)
        
        encoder_output = Dense(units=self.z_size, activation="tanh")(encoder_model)
        
        self.feature_extractor = Model(encoder_input, encoder_output)
        
        return Model(encoder_input, encoder_output)
        
    def build_generator(self):
        # We will map z, a latent vector, to continuous returns data space (..., 5).
        latent = Input(shape=(self.z_size,))

        # This produces a (..., 100) shaped tensor.
        generator_model = Dense(units=100, activation="elu")(latent)
        generator_model = BatchNormalization()(generator_model)
        generator_model = Dense(units=100, activation="elu")(generator_model)
        generator_model = BatchNormalization()(generator_model)

        generator_output = Dense(units=5, activation="linear")(generator_model)
        
        return Model(latent, generator_output)
    
    def build_discriminator(self):
        z = Input(shape=(self.z_size,))
        ret_data = Input(shape=(5,))
        discriminator_inputs = concatenate([z, ret_data], axis=1)

        discriminator_model = Dense(units=100)(discriminator_inputs)
        discriminator_model = LeakyReLU(alpha=0.2)(discriminator_model)
        discriminator_model = Dropout(rate=0.5)(discriminator_model)
        discriminator_output = Dense(units=1, activation="sigmoid")(discriminator_model)
        
        return Model([z, ret_data], discriminator_output)

z_size = 10
bigan = BiGAN(z_size=z_size,
              batch_size=100,
              iterations=10000)

bigan.fit(X=ret_data)

n_sim = 1000
noise = np.random.uniform(low=-1.0, high=1.0, size=(n_sim, z_size))
x = np.zeros(shape=(n_sim, 5))
x_mean = np.zeros(shape=n_sim)
for i, xi in enumerate(noise):
    x[i, :] = (bigan.generator.predict(x=np.array([xi]))[0] * std) + mean
    x_mean[i] = np.average(a=x[i, :])

act_mean = np.zeros(shape=ret_data.shape[0])
for i in range(ret_data.shape[0]):
    act_mean[i] = np.average(a=(ret_data.iloc[i] * std) + mean)
    
plotnine.options.figure_size = (12, 9)
plot = ggplot(pd.melt(pd.concat([pd.DataFrame(x_mean, columns=["BiGAN Portfolio Returns Distribution"]).reset_index(drop=True),
                                 pd.DataFrame(act_mean, columns=["Actual Portfolio Returns Distribution"]).reset_index(drop=True)],
                                axis=1))) + \
geom_density(aes(x="value",
                 fill="factor(variable)"), 
             alpha=0.5,
             color="black") + \
geom_point(aes(x="value",
               y=0,
               fill="factor(variable)"), 
           alpha=0.5, 
           color="black") + \
xlab("Portfolio returns") + \
ylab("Density") + \
ggtitle("Trained Bidirectional Generative Adversarial Network (BiGAN) Portfolio Returns") + \
theme_matplotlib()
plot.save(filename="trained_bigan_sampler.png")

untrained_bigan = BiGAN(z_size=z_size,
                        batch_size=100,
                        iterations=10000)

untrained_x = np.zeros(shape=(n_sim, 5))
untrained_x_mean = np.zeros(shape=n_sim)
for i, xi in enumerate(noise):
    untrained_x[i, :] = (untrained_bigan.generator.predict(x=np.array([xi]))[0] * std) + mean
    untrained_x_mean[i] = np.average(a=untrained_x[i, :])

plotnine.options.figure_size = (12, 9)
plot = ggplot(pd.melt(pd.concat([pd.DataFrame(untrained_x_mean, columns=["BiGAN Portfolio Returns Distribution"]).reset_index(drop=True),
                                 pd.DataFrame(act_mean, columns=["Actual Portfolio Returns Distribution"]).reset_index(drop=True)],
                                axis=1))) + \
geom_density(aes(x="value",
                 fill="factor(variable)"), 
             alpha=0.5,
             color="black") + \
geom_point(aes(x="value",
               y=0,
               fill="factor(variable)"), 
           alpha=0.5, 
           color="black") + \
xlab("Portfolio returns") + \
ylab("Density") + \
ggtitle("Untrained Bidirectional Generative Adversarial Network (BiGAN) Portfolio Returns") + \
theme_matplotlib()
plot.save(filename="untrained_bigan_sampler.png")

print("The VaR at 1%% estimate given by the BiGAN: %.2f%%" % (100 * np.percentile(a=x_mean, axis=0, q=1)))

UnboundLocalError: ignored